In [1]:
from categories import print_tests, all_tests
from rdkit import Chem

[17:15:05] SMARTS Parse Error: syntax error while parsing: [#7](=[#8])$[[#1],[#6X4]]{1..;xm}
[17:15:05] SMARTS Parse Error: Failed parsing SMARTS '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}' for input: '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}'
[17:15:05] SMARTS Parse Error: syntax error while parsing: $[[#1],[#6X4]]{1..;xm}[#7+][#8-]
[17:15:05] SMARTS Parse Error: Failed parsing SMARTS '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]' for input: '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]'
[17:15:05] SMARTS Parse Error: syntax error while parsing: c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}
[17:15:05] SMARTS Parse Error: Failed parsing SMARTS 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}' for input: 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}'
[17:15:05] SMARTS Parse Error: syntax error while parsing: $[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]
[17:15:05] SMARTS Parse Error: Failed parsing SMARTS '$[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]' for input: '$[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]'
[17:15:05] SMARTS Parse Error: syntax error while parsing: [#8]{>-

In [2]:
def subqueryFunction(x):
    """Returns a warning that the subquery query is contained in the parent query function
    for the hard-coded category tests."""
    return "This subquery portion is contained in the parent Logical Query."

def humanBuiltQuery(function = None, query_words = {'qtype':None, 'smart':None, 'prop':None, 'operand':None, 'logic': None, 'subqueries':None}):
    """This function is an attempt to automate building queries for the hard-coded tests below such that
    print_tree functionality should still work with these newer test types. The query_words dictionary MUST contain
    the key 'qtype' with value 'b:StructureQuery', 'b:ParameterQuery', 'b:ExclusionQuery', or 'LogicalQuery'. Other
    keys should contain the necessary information for print_tree for the given qtype. The function will be the
    query function, as built below. 
    
    Required keys by qtype:
    - Structure or Exclusion Query - 'smart'
    - Parameter Query - 'prop', 'operand', 'value'
    - Logical Query - 'logic', 'subqueries': provided as a list of dictionaries obeying input rules for this function
    """
    new_query = Query(query_words)
    new_query.query = function
    qtype = query_words['qtype']
    new_query.type = qtype
    if qtype in ['b:StructureQuery', 'b:ExclusionQuery']:
        new_query.smart = query_words['smart']
    elif qtype == 'b:ParameterQuery':
        new_query.prop = query_words['prop']
        new_query.operand = query_words['operand']
        new_query.value = query_words['value']
    elif qtype == 'LogicalQuery':
        new_query.logic = query_words['logic']
        new_query.subqueries = [humanBuiltQuery(None, query_words['subqueries'][i]) for i in range(len(query_words['subqueries']))]

    return new_query

In [3]:
from categories import Query
def create_test():
    primamine=Chem.MolFromSmarts('[NX3;H2;!$(NC=[O,N,S]);!$(NCN)][CX3]')
    secamine=Chem.MolFromSmarts('[NX3;H1;!$(NC=[O,N,S]);!$(NCN)](C)[CX3]')
    tertamine=Chem.MolFromSmarts('[N;!$(NC=[O,N,S]);!$(NCN)](C)(C)[CX3]')
    def test(x):
        mol=x['mol']
        smiles=x['smiles']
        mw=x['mol_weight']
        return 'c' not in smiles and mw<1000 and '1' not in smiles and (mol.HasSubstructMatch(primamine) or mol.HasSubstructMatch(secamine)\
        or mol.HasSubstructMatch(tertamine)) 
    return test

aa_words = {'qtype':'LogicalQuery', 'logic':'And', \
             'subqueries': [{'qtype':'b:ExclusionQuery', 'smart':'c'}, \
                            {'qtype':'b:ParameterQuery', 'prop':'Molecular Weight', 'operand':'LessThan', 'value': 1000}, \
                            {'qtype': 'LogicalQuery', 'logic':'or', \
                              'subqueries': [{'qtype': 'b:StructureQuery', 'smart':'[NX3;H2;!$(NC=[O,N,S]);!$(NCN)][CX3]'}, \
                                             {'qtype': 'b:StructureQuery', 'smart':'[NX3;H1;!$(NC=[O,N,S]);!$(NCN)](C)[CX3]'}, \
                                             {'qtype': 'b:StructureQuery', 'smart':'[N;!$(NC=[O,N,S]);!$(NCN)](C)(C)[CX3]'}]}]} 
all_tests['Aliphatic Amines']=humanBuiltQuery(create_test(), aa_words)

In [4]:
# Define a test chemical as chemicals are stored in the GenRA database
test_chem = {'dsstox_sid': 'DTXSID7020009',
 'smiles': 'CC#N',
 'logp': -0.33971,
 'ws': 12.6417,
 'mol_weight': 41.053,
 'mol': Chem.MolFromSmiles('CC#N')}

In [5]:
from categories import printTree

printTree('Acid Chlorides', test_chem)

('32', 'LogicalQuery', 'And', False)
	('29', 'b:ParameterQuery', 'Molecular weight', 1000.0, 'LessThan', True)
	('26', 'b:ParameterQuery', 'log Kow', 8.0, 'LessThan', True)
	('24', 'b:StructureQuery', '[#6,#16](=[#8])(Cl)[#6X4,c]', False)


In [6]:
all_tests['Aliphatic Amines'].query(test_chem)

False

In [7]:
all_tests['Aliphatic Amines'].print_tree(None)

(None, 'LogicalQuery', 'And', 'does not process')
	(None, 'b:ExclusionQuery', 'does not process')
	(None, 'b:ParameterQuery', 'Molecular Weight', 1000, 'LessThan', 'does not process')
	(None, 'LogicalQuery', 'or', 'does not process')
		(None, 'b:StructureQuery', '[NX3;H2;!$(NC=[O,N,S]);!$(NCN)][CX3]', 'does not process')
		(None, 'b:StructureQuery', '[NX3;H1;!$(NC=[O,N,S]);!$(NCN)](C)[CX3]', 'does not process')
		(None, 'b:StructureQuery', '[N;!$(NC=[O,N,S]);!$(NCN)](C)(C)[CX3]', 'does not process')


In [8]:
all_tests['Polynitroaromatics (Chronic toxicity)'].print_tree(test_chem)

('CustomQuery', 'LogicalQuery', 'And', False)
	('CustomQuery', 'b:StructureQuery', 'N[$(c1c(N)cccc1),$(c1cc(N)ccc1),$(c1ccc(N)cc1),$(c1cncc(N)c1)]', 'does not process')
	('CustomQuery', 'b:ParameterQuery', 'Molecular Weight', 1000, 'LessThan', 'does not process')
	('CustomQuery', 'b:ParameterQuery', 'Log Kow', 10, 'GreaterThanOrEqualTo', 'does not process')


In [9]:
the_keys = list(all_tests.keys())
the_keys.sort()
for key in the_keys:
    print('- ' + key)

- Acid Chlorides
- Acrylamides
- Acrylates/Methacrylates (Acute toxicity)
- Acrylates/Methacrylates (Chronic toxicity)
- Aldehydes (Acute toxicity)
- Aldehydes (Chronic toxicity)
- Aliphatic Amines
- Alkoxysilanes
- Aluminum Compounds
- Aminobenzothiazole Azo Dyes
- Anhydrides, Carboxylic acid
- Anilines (Acute toxicity)
- Anilines (Chronic toxicity)
- Anionic Surfactants
- Azides (Acute toxicity)
- Azides (Chronic toxicity)
- Benzotriazole-hindered phenols
- Benzotriazoles
- Benzotriazoles (Acute toxicity)
- Benzotriazoles (Chronic toxicity)
- Boron Compounds
- Cationic (quaternary ammonium) surfactants
- Cobalt
- Dianilines
- Diazoniums (Acute toxicity)
- Diazoniums (Chronic toxicity)
- Dichlorobenzidine-based Pigments
- Diisocyanates
- Dithiocarbamates (Acute toxicity)
- Dithiocarbamates (Chronic toxicity)
- Epoxides
- Esters (Acute toxicity)
- Esters (Chronic toxicity)
- Ethylene Glycol Ethers
- Hindered Amines
- Hydrazines and Related Compounds
- Imides (Acute toxicity)
- Imides (

In [10]:
all_tests['Acid Chlorides'].print_tree

<bound method Query.print_tree of <categories.Query object at 0x0000022561D1BBF0>>

In [11]:
the_test_dictionary = {}
for key in all_tests.keys():
    the_test_dictionary[key] = printTree(key,None, printer = False)

In [13]:
printTree('Boron Compounds', None)

('230', 'LogicalQuery', 'And', 'does not process')
	('225', 'b:StructureQuery', '[#5]', 'does not process')
	('227', 'b:ParameterQuery', 'Molecular weight', 1000.0, 'LessThan', 'does not process')


In [23]:
smarts=[]
match_mols=[]
prefix='[NH2]c1[cH1,$(cO)]'
suffix='[cH][cH]1'
for c1 in range(1,4):
    for c2 in range(c1+1,5):
        smarts.append(prefix+'c2'+'[cH]'*(c1-1)+'[cH,$(c[OH]),$(c[NH2])]'+'[cH]'*(c2-c1-1)+'c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])'+'[cH]'*(4-c2)+'c2'+suffix)
        smarts.append(prefix+'c2'+'[cH]'*(c1-1)+'c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])'+'[cH]'*(c2-c1-1)+'[cH1,$(c[OH]),$(c[NH2])]'+'[cH]'*(4-c2)+'c2'+suffix)
smarts

['[NH2]c1[cH1,$(cO)]c2[cH,$(c[OH]),$(c[NH2])]c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])[cH][cH]c2[cH][cH]1',
 '[NH2]c1[cH1,$(cO)]c2c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])[cH1,$(c[OH]),$(c[NH2])][cH][cH]c2[cH][cH]1',
 '[NH2]c1[cH1,$(cO)]c2[cH,$(c[OH]),$(c[NH2])][cH]c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])[cH]c2[cH][cH]1',
 '[NH2]c1[cH1,$(cO)]c2c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])[cH][cH1,$(c[OH]),$(c[NH2])][cH]c2[cH][cH]1',
 '[NH2]c1[cH1,$(cO)]c2[cH,$(c[OH]),$(c[NH2])][cH][cH]c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])c2[cH][cH]1',
 '[NH2]c1[cH1,$(cO)]c2c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])[cH][cH][cH1,$(c[OH]),$(c[NH2])]c2[cH][cH]1',
 '[NH2]c1[cH1,$(cO)]c2[cH][cH,$(c[OH]),$(c[NH2])]c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])[cH]c2[cH][cH]1',
 '[NH2]c1[cH1,$(cO)]c2[cH]c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])[cH1,$(c[OH]),$(c[NH2])][cH]c2[cH][cH]1',
 '[NH2]c1[cH1,$(cO)]c2[cH][cH,$(c[OH]),$(c[NH2])][cH]c([$(S(

In [24]:
# Define a test chemical as chemicals are stored in the GenRA database
test_chem = {'dsstox_sid': 'DTXSID7020009',
 'smiles': 'CC#N',
 'logp': -0.33971,
 'ws': 12.6417,
 'mol_weight': 41.053,
 'mol': Chem.MolFromSmiles('CC#N')}

In [25]:
from categories import queryAll

In [26]:
print_tests

{'Acid Chlorides': ['32', 'LogicalQuery'],
 'Acrylamides': ['51', 'LogicalQuery'],
 'Acrylates/Methacrylates (Acute toxicity)': ['70', 'LogicalQuery'],
 'Aldehydes (Acute toxicity)': ['89', 'LogicalQuery'],
 'Aliphatic Amines': ['121', 'LogicalQuery'],
 'Aluminum Compounds': ['140', 'LogicalQuery'],
 'Anilines (Acute toxicity)': ['159', 'LogicalQuery'],
 'Azides (Acute toxicity)': ['184', 'LogicalQuery'],
 'Benzotriazoles (Acute toxicity)': ['203', 'LogicalQuery'],
 'Benzotriazole-hindered phenols': ['214', 'b:StructureQuery'],
 'Boron Compounds': ['230', 'LogicalQuery'],
 'Cationic (quaternary ammonium) surfactants': ['241', 'b:StructureQuery'],
 'Cobalt': ['263', 'LogicalQuery'],
 'Diazoniums (Acute toxicity)': ['282', 'LogicalQuery'],
 'Epoxides': ['298', 'LogicalQuery'],
 'Esters (Acute toxicity)': ['317', 'LogicalQuery'],
 'Hydrazines and Related Compounds': ['336', 'LogicalQuery'],
 'Hindered Amines': ['350', 'b:StructureQuery'],
 'Imides (Acute toxicity)': ['369', 'LogicalQuery'

In [27]:
new_test_chems = [{'dsstox_sid': 'DTXSID3060164',
  'smiles': 'C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1',
  'logp': 5.76,
  'ws': 4.07380277804113e-07,
  'mol_weight': 244.125200512,
  'mol': Chem.MolFromSmiles('C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1')},
 {'dsstox_sid': 'DTXSID7060837',
  'smiles': 'ICCCI',
  'logp': 3.02,
  'ws': 0.0007413102413009177,
  'mol_weight': 295.855896192,
  'mol': Chem.MolFromSmiles('ICCCI')},
 {'dsstox_sid': 'DTXSID9025879',
  'smiles': 'OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1',
  'logp': 1.99,
  'ws': 0.009120108393559097,
  'mol_weight': 218.0579088,
  'mol': Chem.MolFromSmiles('OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1')},
 {'dsstox_sid': 'DTXSID2026282',
  'smiles': 'O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=2C4=O)NC(=O)C2C=CC=CC=2)C2=C1C(=O)C1C=CC=CC=1C2=O)C1C=CC=CC=1',
  'logp': 3.11,
  'ws': 2.454708915685029e-08,
  'mol_weight': 667.174335520001,
  'mol': Chem.MolFromSmiles('O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=2C4=O)NC(=O)C2C=CC=CC=2)C2=C1C(=O)C1C=CC=CC=1C2=O)C1C=CC=CC=1')},
 {'dsstox_sid': 'DTXSID4052188',
  'smiles': 'CC1(C)COC(C)(OC1)C1C=CC=CC=1',
  'logp': 2.72,
  'ws': 0.001148153621496883,
  'mol_weight': 206.130679816,
  'mol': Chem.MolFromSmiles('CC1(C)COC(C)(OC1)C1C=CC=CC=1')}]

In [28]:
import pandas as pd
pd.DataFrame(new_test_chems).drop(columns = ['mol']).to_csv('data/raw/readme_examples.csv')

In [29]:
test_chems_df = pd.read_csv('data/raw/readme_examples.csv').set_index('Unnamed: 0', drop = True)
test_chems_df['mols'] = [Chem.MolFromSmiles(smile) for smile in test_chems_df['smiles']]
test_chems_df

,dsstox_sid,smiles,logp,ws,mol_weight,mols
Unnamed: 0,,,,,,
0,DTXSID3060164,C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1,5.76,4.073803e-07,244.125201,<rdkit.Chem.rdchem.Mol object at 0x0000025E508...
1,DTXSID7060837,ICCCI,3.02,7.413102e-04,295.855896,<rdkit.Chem.rdchem.Mol object at 0x0000025E508...
2,DTXSID9025879,OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1,1.99,9.120108e-03,218.057909,<rdkit.Chem.rdchem.Mol object at 0x0000025E508...
3,DTXSID2026282,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,3.11,2.454709e-08,667.174336,<rdkit.Chem.rdchem.Mol object at 0x0000025E508...
4,DTXSID4052188,CC1(C)COC(C)(OC1)C1C=CC=CC=1,2.72,1.148154e-03,206.130680,<rdkit.Chem.rdchem.Mol object at 0x0000025E508...


In [30]:
df = queryAll(new_test_chems[0])
for category in all_tests.keys():
     for chem in df.index:
          if df.loc[chem,category] == True:
               print(category)

Neutral Organics


In [31]:
all_tests['Acid Chlorides'].subqueries

In [32]:
error_test = {'dsstox_sid': 'DTXSID3060164',
  'smiles': 'C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1',
  'mol_weight': 'A',
  'mol': Chem.MolFromSmiles('C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1')}

In [33]:
queryAll(error_test)

KeyError: logp must be provided
KeyError: ws must be provided
TypeError: mol_weight must be provided as float64. Please adjust the input accordingly.


KeyError: 'One or more attributes is missing. See printed statement(s).'

In [34]:
queryAll(new_test_chems)

,chemicals,Acid Chlorides,Acrylamides,Acrylates/Methacrylates (Acute toxicity),Aldehydes (Acute toxicity),Aliphatic Amines,Aluminum Compounds,Anilines (Acute toxicity),Azides (Acute toxicity),Benzotriazoles (Acute toxicity),...,Organotins (Chronic toxicity),Phenols (Chronic toxicity),Phosphinate Esters (Chronic toxicity),Polynitroaromatics (Chronic toxicity),Substituted Triazines (Chronic toxicity),Thiols (Chronic toxicity),Vinyl Esters (Chronic toxicity),Diazoniums (Chronic toxicity),Ethylene Glycol Ethers,Benzotriazoles
0,DTXSID3060164,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,DTXSID7060837,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,DTXSID9025879,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,DTXSID2026282,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,DTXSID4052188,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [35]:
import pandas as pd
mini_test_chem = dict(pd.DataFrame(new_test_chems).iloc[0])

from categories import checkForAttributes, normalizeChemicals, listCategories
mini_test_chem = normalizeChemicals(mini_test_chem)
checkForAttributes(mini_test_chem)
queryAll(mini_test_chem)

dict(mini_test_chem)
listCategories(mini_test_chem)

['Neutral Organics']

In [43]:
chems_in_categories = [{'dsstox_sid':'DTXSID90480751', 'smiles':'C[13C](Cl)=O', 
                        'mol_weight':79.49, 'ws':1.00, 'mol':Chem.MolFromSmiles('C[13C](Cl)=O'), 
                        'logp':0.380},
                        {'dsstox_sid':'DTXSID50939730', 'smiles':'CC(=C)C([O-])=O', 
                        'mol_weight':85.083, 'ws':0.704, 'mol':Chem.MolFromSmiles('CC(=C)C([O-])=O'), 
                        'logp':0.857},
                        {'dsstox_sid':'DTXSID2036405', 'smiles':'[OH-].[OH-].[OH-].[Al+3]', 
                         'mol_weight':78.003, 'ws':28.8, 'mol':Chem.MolFromSmiles('[OH-].[OH-].[OH-].[Al+3]'),
                         'logp':-0.750},
                         {'dsstox_sid':'DTXSID1024835', 'smiles':'O=CC=CC1=CC=CC=C1',
                          'mol_weight':132.162, 'ws':1.07*(10**-2), 'mol':Chem.MolFromSmiles('O=CC=CC1=CC=CC=C1'),
                          'logp':1.90},
                          {'dsstox_sid':'DTXSID30878870', 'smiles':'[N-]=[N+]=NC1=CC=CC=C1',
                           'mol_weight':119.127, 'ws':2.18*(10**-2), 'mol':Chem.MolFromSmiles('[N-]=[N+]=NC1=CC=CC=C1'),
                           'logp':2.59}]

queryAll(chems_in_categories)

,chemicals,Acid Chlorides,Acrylamides,Acrylates/Methacrylates (Acute toxicity),Aldehydes (Acute toxicity),Aliphatic Amines,Aluminum Compounds,Anilines (Acute toxicity),Azides (Acute toxicity),Benzotriazoles (Acute toxicity),...,Organotins (Chronic toxicity),Phenols (Chronic toxicity),Phosphinate Esters (Chronic toxicity),Polynitroaromatics (Chronic toxicity),Substituted Triazines (Chronic toxicity),Thiols (Chronic toxicity),Vinyl Esters (Chronic toxicity),Diazoniums (Chronic toxicity),Ethylene Glycol Ethers,Benzotriazoles
0,DTXSID90480751,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,DTXSID50939730,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,DTXSID2036405,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,DTXSID1024835,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,DTXSID30878870,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
